In [1]:
import nltk,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import nltk.data
import csv
from nltk.stem.porter import *
pd.options.mode.chained_assignment = None
from sklearn.preprocessing import LabelEncoder
from numpy import array as arr
from numpy import argmax

In [2]:
arr=[]
file = open("downloaded1.csv",'rt')
samples=csv.reader(file)
c=0
for i in samples:
    c+=1
    
    if c==2:
        x=i[1]
        break

for i in samples:
    if i[1]!=x:
        arr.append(i)

data=arr
df=pd.DataFrame(data=arr,columns=("types","posts"))

In [3]:
def labelencode(df):
    data=df['types']
    values=np.array(data)
    label=LabelEncoder()
    intencode=label.fit_transform(values)
    df['typeint']=intencode
    print(list(label.inverse_transform([0,1,2])))
    #df['typeint'].plot(kind='hist')
    #k=np.arange(0,16)
    #x=label.inverse_transform(k)   #can access encoded actual value using x
    #print(values)
    return df

df=labelencode(df)
#print(df)
shortdata=df.iloc[:,1]

['negative', 'neutral', 'positive']


In [4]:
#removing stopwords 
from nltk.corpus import stopwords
stop=stopwords.words("english")
print('------Removing stopwords------')
shortdata=shortdata.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#shortdata=shortdata.apply(lambda x: ' '.join([word for word in x.split() if word!='i' or word!='I']))
#print(shortdata)
#stemming of words
ps = PorterStemmer()
print('-------Stemming--------')
shortdata = shortdata.apply(lambda x: ' '.join([ps.stem(word) for word in x.split() ]))
#print(shortdata)

------Removing stopwords------
-------Stemming--------


In [5]:
#removing non-alphabets
shortdata=shortdata.apply(lambda x: ' '.join([word for word in x.split() if word.isalpha()]))

from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
#print(shortdata)
print('-------Lemmatization--------')
shortdata = shortdata.apply(lambda x: ' '.join([lmtzr.lemmatize(word,'v') for word in x.split() ]))
#print(shortdata)

print('--------Removing punctuations--------')
def clear_punctuation(s):
	import string
	#print("\n")
	clear_string = ""
	for symbol in s:
		if symbol not in string.punctuation:
			clear_string += symbol
	return clear_string

shortdata = shortdata.apply(lambda x: ''.join(clear_punctuation(x))  )
#print(shortdata)
#for line in shortdata:
#	print(line)
#	print('\n')


-------Lemmatization--------
--------Removing punctuations--------


In [6]:
def strip_all_entities(text):
	import string
	entity_prefixes = ['@']
	for separator in  string.punctuation:
		if separator not in entity_prefixes :
			text = text.replace(separator,' ')
	words = []
	for word in text.split():
		word = word.strip()
		if word:
			if word[0] not in entity_prefixes:
				words.append(word)
	return ' '.join(words)

shortdata = shortdata.apply(lambda x: ''.join(strip_all_entities(x))  ) 



In [7]:
i=0
arr=[]
print("-----PREPROCESSED_DATA------")
count=0
for line in shortdata:
    df.iloc[i,1]=line
    i=i+1
#print(df)

-----PREPROCESSED_DATA------


In [18]:
newdf=pd.DataFrame(data=data,columns=("types","posts"))
newdf=labelencode(newdf)

['negative', 'neutral', 'positive']


In [14]:
nposts=np.array(newdf['posts'])
nlabels=np.array(newdf['typeint'])
print(len(nposts))
print(len(nlabels))

6936
6936


In [19]:
import tensorflow

ModuleNotFoundError: No module named 'tensorflow'

In [23]:
!pip install tensorflow

  Could not find a version that satisfies the requirement tensorflow (from versions: )
No matching distribution found for tensorflow


In [ ]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

In [ ]:
newdf['posts'] = newdf['posts'].apply(lambda x: x.lower())

newdf['posts'] = newdf['posts'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


In [ ]:
max_fatures = 5000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(newdf['posts'].values)
X = tokenizer.texts_to_sequences(newdf['posts'].values)
X = sequence.pad_sequences(X)

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
#model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
Y = pd.get_dummies(newdf['types']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

In [ ]:
score,acc=(model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size))
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))